# Enable GPU acceleration
Go to: 
`Edit > Notebook settings > Hardware accelerator > GPU`

In [5]:
import tensorflow as tf
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import urllib.request
import tarfile

print("Libraries version:")
print(f"Numpy:      {np.__version__}")
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Tensorflow: {tf.__version__}")

Libraries version:
Numpy:      1.18.5
Matplotlib: 3.2.2
Tensorflow: 2.2.0


# Get all files

In [6]:
%cd /content

/content


## Load Git repo

In [7]:
if not os.path.isdir("AutonomousRcCar"):
    !git clone https://github.com/maximecharriere/AutonomousRcCar.git

Cloning into 'AutonomousRcCar'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 1059 (delta 2), reused 11 (delta 2), pack-reused 1047
Receiving objects: 100% (1059/1059), 554.52 MiB | 9.62 MiB/s, done.
Resolving deltas: 100% (386/386), done.
Checking out files: 100% (558/558), done.


In [19]:
model_training_dir_path = "/content/AutonomousRcCar/autonomouscar/tf_model_training"

In [8]:
if not os.path.isdir("models"):
    !git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 38032, done.
remote: Total 38032 (delta 0), reused 0 (delta 0), pack-reused 38032
Receiving objects: 100% (38032/38032), 544.52 MiB | 25.20 MiB/s, done.
Resolving deltas: 100% (25503/25503), done.


## Load pre-trained model

Choose a tensorflow model:
- [All object detection models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models)
- [Models compatible with Coral Edge TPU](https://coral.ai/models/)

In [9]:
model_name = 'ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03'

if not os.path.isdir(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    urllib.request.urlretrieve(base_url + model_file, model_file)
    with tarfile.open(model_file) as tar:
        tar.extractall()
    os.remove(model_file)


In [10]:
fine_tune_checkpoint = os.path.join(model_name, "model.ckpt")

# Install the Tensorflow Object Detection API
It's necessary in the next scripts

In [11]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
!pip install -q tf_slim

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

!python object_detection/builders/model_builder_test.py

Selecting previously unselected package python-bs4.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

# Generate `tfrecord` files

In [27]:
%cd {model_training_dir_path}

/content/AutonomousRcCar/autonomouscar/tf_model_training


In [13]:
#Partition data between test and train set
!python3 scripts/partition_dataset.py -i training/images/all -o training/images -r 0.2 -x

In [14]:
# Convert train folder annotation xml files to a single csv file,
# generate the `label_map.pbtxt` file to `data/` directory as well.
!python3 scripts/xml_to_csv.py -i training/images/train -o training/annotations/train_labels.csv -l training/annotations

# Convert test folder annotation xml files to a single csv.
!python3 scripts/xml_to_csv.py -i training/images/test -o training/annotations/test_labels.csv

Successfully converted xml to csv.
Generate `training/annotations/label_map.pbtxt`
Successfully converted xml to csv.


In [18]:
# Generate `train.record`
!python scripts/generate_tfrecord.py --csv_input=training/annotations/train_labels.csv --output_path=training/annotations/train.record --img_path=training/images/train --label_map training/annotations/label_map.pbtxt

# Generate `test.record`
!python scripts/generate_tfrecord.py --csv_input=training/annotations/test_labels.csv --output_path=training/annotations/test.record --img_path=training/images/test --label_map training/annotations/label_map.pbtxt


2020-07-02 14:55:28.227166: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/AutonomousRcCar/autonomouscar/tf_model_training/training/annotations/train.record
2020-07-02 14:55:32.690780: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Successfully created the TFRecords: /content/AutonomousRcCar/autonomouscar/tf_model_training/training/annotations/test.record


# Configuring the Training `pipeline.conf`

In [ ]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [22]:
batch_size = 24
num_steps = 1000
num_classes = get_num_classes(label_map_pbtxt_fname)
test_record_fname = model_training_dir_path + '/training/annotations/test.record'
train_record_fname = model_training_dir_path + '/training/annotations/train.record'
label_map_pbtxt_fname = model_training_dir_path + '/training/annotations/label_map.pbtxt'

In [29]:
%cd /content
pipeline_fname = os.path.join(model_name, "pipeline.config")
assert os.path.isfile(pipeline_fname), f'`{pipeline_fname}` not exist'

/content


In [ ]:
import re


with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               f'fine_tune_checkpoint: "{fine_tune_checkpoint}"', s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', f'input_path: "{train_record_fname}"', s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', f'input_path: "{test_record_fname}"', s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', f'label_map_path: "{label_map_pbtxt_fname}"', s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               f'batch_size: {batch_size}', s)
    
    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               f'num_steps: {num_steps}', s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               f'num_classes: {num_classes}', s)
    f.write(s)